# Setup section

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
# Download repositories

!sudo rm -dr GPUcomputing
!git clone https://github.com/S3gmentati0nFault/GPUcomputing.git
!mkdir testing

In [ ]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!python3 -m build
%load_ext nvcc4jupyter
%cd /content/

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)

!nvcc -arch=sm_75 /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery

In [ ]:
# Installation of the Valgrind utility

!sudo apt install valgrind

# Configuration

Test interessanti:

SIZE 20000

MAX_WEIGHT 50000

FIXED_SEED 78651423

In [ ]:
%%cuda_group_save --name "sharedMacros.h" --group "COMMON"

#ifndef SHARED_MACROS_H
#define SHARED_MACROS_H

#define DEBUGGING 0
#define SIZE 20000
#define MAX_WEIGHT 50000
#define TESTING 0
#define TEST_SIZE 3
#define FIXED_SEED 78651423
#define BLOCK_SIZE 1024

#endif

# CPU zone

## MST solution with Prim

In [ ]:
%%cuda_group_save --name "mst.h" --group "CPU"

#ifndef MST_H
#define MST_H

// Header file di C++
#include <vector>

struct mst {
    std::vector<int> *stree;
    int totalWeight;

    mst() {
        stree = NULL;
        totalWeight = 0;
    }

    ~mst() {
        if (stree != NULL) {
            delete[] stree;
        }
        stree = NULL;
        totalWeight = 0;
    }
};

#endif

### Heap implementation

In [ ]:
%%cuda_group_save --name "heap.h" --group "CPU"

// Header file di C++
#include <vector>

// Header file custom
#include "../COMMON/sharedMacros.h"

using namespace std;

#ifndef HEAP_H
#define HEAP_H

struct edge {
    edge () {
        source = UINT_MAX;
        destination = UINT_MAX;
        offset = UINT_MAX;
        weight = INT_MAX;
    }

    ~edge () {
        source = UINT_MAX;
        destination = UINT_MAX;
        offset = UINT_MAX;
        weight = INT_MAX;
    }

    edge (uint source, uint destination, uint offset, int weight) {
        this->source = source;
        this->destination = destination;
        this->offset = offset;
        this->weight = weight;
    }

    bool operator<(const edge& other) const {
        if (this->weight == other.weight) {
            return this->destination < other.destination;
        }
        return this->weight < other.weight;
    }

    bool operator>(const edge& other) const {
        if (this->weight == other.weight) {
            return this->destination > other.destination;
        }
        return this->weight > other.weight;
    }

    uint source, destination, offset;
    int weight;
};

class Heap {
    public:
        Heap () {
            size = 0;
            map = NULL;
        }

        Heap (uint size) {
            mapSize = size;
            map = new uint[mapSize];
            for (uint i = 0; i < mapSize; i++) {
                map[i] = UINT_MAX;
            }
            this->size = 0;
        }

        ~Heap () {
            delete[] map;
            map = NULL;
        }

        uint getLeftChild (uint index) {
            return 2 * index + 1;
        }

        uint getRightChild (uint index) {
            return 2 * index + 2;
        }

        uint getParent (uint index) {
            return (index - 1) / 2;
        }

        edge getKey (uint index) {
            return heap[index];
        }

        uint getPosition (uint key) {
            return map[key];
        }

        void insert (uint source, uint destination, uint offset, int weight) {
            // Insert the new edge
            edge newEdge = edge(source, destination, offset, weight);
            heap.push_back(newEdge);
            size++;

            map[destination] = size - 1;

            uint i = size - 1;
            while (i > 0 && heap[getParent(i)] > heap[i]) {
                swap(heap[i], heap[getParent(i)]);
                swap(map[heap[i].destination], map[heap[getParent(i)].destination]);
                i = getParent(i);
            }
        }

        void heapify (uint subtree) {
            uint left = getLeftChild(subtree);
            uint right = getRightChild(subtree);
            uint minimum = subtree;

            if (left < size && heap[left] < heap[minimum]) {
                minimum = left;
            }

            if (right < size && heap[right] < heap[minimum]) {
                minimum = right;
            }

            if (minimum != subtree) {
                swap(heap[subtree], heap[minimum]);
                swap(map[heap[subtree].destination], map[heap[minimum].destination]);

                heapify(minimum);
            }
        }

        void print() {
            for (uint i = 0; i < size; i++) {
                cout << "(" << heap[i].source << ", " << heap[i].destination << ")[" << heap[i].weight << "]" << endl;
            }
        }

        void printMap() {
            for (uint i = 0; i < mapSize; i++) {
                cout << i << ", " << map[i] << endl;
            }
        }

        edge peek() {
            return heap[0];
        }

        edge pop() {
            edge min;
            if (size == 0) {
                return min;
            }

            swap(heap[0], heap[size - 1]);
            swap(map[heap[0].destination], map[heap[size - 1].destination]);
            min = heap[size - 1];

            heap.erase(heap.end());
            map[min.destination] = UINT_MAX;
            size--;

            if (size == 0) {
                return min;
            }

            heapify(0);
            return min;
        }

        bool isEmpty() {
            if (size == 0) {
                return true;
            }
            return false;
        }

        void keyDecrease(uint i, uint source, int weight) {
            heap[i].weight = weight;
            heap[i].source = source;

            while (i > 0 && heap[getParent(i)] > heap[i]) {
                swap(heap[i], heap[getParent(i)]);
                swap(map[heap[i].destination], map[heap[getParent(i)].destination]);
                i = getParent(i);
            }
        }

    private:
        uint size, mapSize;
        uint *map;
        vector<edge> heap;
};

#endif

In [ ]:
%%cuda_group_save --name "heap.cu" --group "CPU"

#include <iostream>

#include "heap.h"

int main() {
    Heap *heap = new Heap();

    // void insert (uint source, uint destination, uint offset, int weight) {
    heap->insert(3, 0, 0, 10);
    heap->insert(3, 1, 0, 11);
    heap->insert(3, 2, 0, 12);
    heap->insert(3, 3, 0, 123);
    heap->insert(3, 4, 0, 14);
    heap->insert(3, 5, 0, 10);
    heap->insert(3, 6, 0, 16);
    heap->insert(3, 7, 0, 10);
    heap->insert(3, 8, 0, 18);
    heap->insert(3, 9, 0, 0);

    heap->print();

    printf("\n\n");

    heap->pop();

    heap->print();

    delete heap;

    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/heap.cu -o heap
!./heap

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/heap.cu -o heap
!valgrind ./heap

### Valgrind testing

In [ ]:
%%cuda_group_save --name "nonTimedMstCPUPrim.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "mst.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Initialize the tree to the emptyset
    mst *tree = new mst;
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Initialize the vertex set U to a random vertex in the graph
    vector<node> U;
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;
    U.push_back(randV);
    for (node i = 0; i < size; i++) {
        if (i != randV) {
            Remaining.push_back(i);
        }
    }

    // While the set U is different from the set V
    while (!Remaining.empty()) {

      /*
       * Find the edge with the lowest weight in the adjacency list of the
       * nodes currently in U
      */
      int minimum = INT_MAX;
      int sourceCandidate = -1;
      int destinationCandidate = -1;
      for (node i = 0; i < U.size(); i++) {
          for (uint j = 0; j < str->deg(U[i]); j++) {
              node neigh = str->getNeigh(U[i], j);
              uint occ = count(U.begin(), U.end(), neigh);
              if (occ == 0 && (str->getWeight(U[i], j) < minimum ||
                  (str->getWeight(U[i], j) == minimum && neigh < destinationCandidate))) {
                  minimum = str->getWeight(U[i], j);
                  sourceCandidate = U[i];
                  destinationCandidate = neigh;
              }
          }
      }

      if (sourceCandidate != -1) {
          if (DEBUGGING) {
              cout << "Source: " << sourceCandidate << " Destination: " << destinationCandidate << " Weight: " << minimum << endl;
          }

          // Add the newfound edge to the mst
          tree->stree[sourceCandidate].push_back(destinationCandidate);
          tree->totalWeight += minimum;

          // Add the destination vertex to the set U
          U.push_back(destinationCandidate);

          // Remove the element using erase function and iterators
          auto it = find(Remaining.begin(), Remaining.end(), destinationCandidate);

          // If element is found found, erase it
          if (it != Remaining.end()) {
              Remaining.erase(it);
          }
      }

      else {
          cout << "CPU-MST [Error]: The graph appears to be disconnected" << endl;
          return tree;
      }
    }

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    char path[] = "/content/testing/primTest.txt";
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }

        cout << "Computing the MST solution for a graph of size " << size << endl;
        //cudaEventRecord(start);
        mst *tree = primMST(size, variableEng, graph->getStruct());
        if (size < 15) {
            cout << "MST solution" << endl;
            for (int i = 0; i < size; ++i) {
                cout << i << ": ";
                for (int j = 0; j < tree->stree[i].size(); ++j) {
                    cout << tree->stree[i][j] << " ";
                }
                cout << endl;
            }
        }
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,\n", size, tree->totalWeight);
        }

        // Memory deallocation
        delete tree;
        delete graph;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    return 0;
}

In [ ]:
# Valgrind testing

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/nonTimedMstCPUPrim.cu -o valgrindCPU
!valgrind ./valgrindCPU

### Simple run

In [ ]:
%%cuda_group_save --name "mstCPUPrim.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "mst.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Initialize the tree to the emptyset
    mst *tree = new mst();
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Initialize the vertex set U to a random vertex in the graph
    vector<node> U;
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;
    U.push_back(randV);
    for (node i = 0; i < size; i++) {
        if (i != randV) {
            Remaining.push_back(i);
        }
    }

    // While the set U is different from the set V
    while (!Remaining.empty()) {

      /*
       * Find the edge with the lowest weight in the adjacency list of the
       * nodes currently in U
      */
      int minimum = INT_MAX;
      int sourceCandidate = -1;
      int destinationCandidate = -1;
      for (node i = 0; i < U.size(); i++) {
          for (uint j = 0; j < str->deg(U[i]); j++) {
              node neigh = str->getNeigh(U[i], j);
              uint occ = count(U.begin(), U.end(), neigh);
              if (occ == 0 && (str->getWeight(U[i], j) < minimum ||
                  (str->getWeight(U[i], j) == minimum && neigh < destinationCandidate))) {
                  minimum = str->getWeight(U[i], j);
                  sourceCandidate = U[i];
                  destinationCandidate = neigh;
              }
          }
      }

      if (sourceCandidate != -1) {
          if (DEBUGGING) {
              cout << "Source: " << sourceCandidate << " Destination: " << destinationCandidate << " Weight: " << minimum << endl;
          }

          // Add the newfound edge to the mst
          tree->stree[sourceCandidate].push_back(destinationCandidate);
          tree->totalWeight += minimum;

          // Add the destination vertex to the set U
          U.push_back(destinationCandidate);

          // Remove the element using erase function and iterators
          auto it = find(Remaining.begin(), Remaining.end(), destinationCandidate);

          // If element is found found, erase it
          if (it != Remaining.end()) {
              Remaining.erase(it);
          }
      }

      else {
          cout << "CPU-MST [Error]: The graph appears to be disconnected" << endl;
          return tree;
      }
    }

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    char path[] = "/content/testing/cpuTest.txt";
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }

        cout << "Computing the MST solution for a graph of size " << size << endl;
        cudaEventRecord(start);
        mst *tree = primMST(size, variableEng, graph->getStruct());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        float milliseconds;
        cudaEventElapsedTime(&milliseconds, start, stop);
        float CPUtime = milliseconds / 1000.0;
        cout << "Time elapsed for CPU computation: " << CPUtime << endl;
        if (size < 15) {
            cout << "MST solution" << endl;
            for (int i = 0; i < size; ++i) {
                cout << i << ": ";
                for (int j = 0; j < tree->stree[i].size(); ++j) {
                    cout << tree->stree[i][j] << " ";
                }
                cout << endl;
            }
        }
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,%f,\n", size, tree->totalWeight, CPUtime);
        }

        // Memory deallocation
        delete tree;
        delete graph;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));
    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/mstCPUPrim.cu -o mstCPUPrim
!./mstCPUPrim

## MST solution using the Prim solver with binary heap

In [ ]:
%%cuda_group_save --name "CPUEfficientMst.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"
#include "mst.h"
#include "heap.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Setup the mst
    mst *tree = new mst();
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Select a starting node
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;

    // Initializing the heap structure
    Heap *heap = new Heap(size);
    for (int i = 0; i < size; ++i) {
        int offset = str->isNeighbor(randV, i);
        if (offset >= 0) {
            heap->insert(randV, i, offset, str->getWeight(randV, offset));
        }
        else if (i != randV) {
            heap->insert(UINT_MAX, i, UINT_MAX, INT_MAX);
        }
    }

    // While the heap is not empty
    while (!heap->isEmpty()) {
        edge candidateEdge = heap->pop();
        node destinationCandidate = candidateEdge.destination;

        for (uint i = 0; i < str->deg(destinationCandidate); i++) {
            node neigh = str->getNeigh(destinationCandidate, i);
            node weight = str->getWeight(destinationCandidate, i);
            uint pos = heap->getPosition(neigh);
            bool inMst = false;

            if (pos == UINT_MAX) {
                inMst = true;
            }

            if (!inMst && weight < heap->getKey(pos).weight) {
                heap->keyDecrease(pos, destinationCandidate, weight);
            }
        }

        // Add the newfound edge to the mst
        tree->totalWeight += candidateEdge.weight;
        tree->stree[candidateEdge.source].push_back(candidateEdge.destination);
    }

    delete heap;
    heap = NULL;

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    char path[] = "/content/testing/primTest.txt";
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }
        cudaEventRecord(start);
        cout << "Computing the MST solution for a graph of size " << size << endl;
        mst *tree = primMST(size, variableEng, graph->getStruct());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        float milliseconds;
        cudaEventElapsedTime(&milliseconds, start, stop);
        float CPUtime = milliseconds / 1000.0;
        cout << "Time elapsed for CPU computation: " << CPUtime << endl;
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,\n", size, tree->totalWeight);
        }

        // Memory deallocation
        delete graph;
        delete tree;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/CPUEfficientMst.cu -o CPUEfficientMst
!./CPUEfficientMst

# GPU zone

## Hybrid approach

The code below uses an hybrid approach where a part of the code is working with the GPU and a part of the code is using the CPU, namely:

- Finding the cheapest edge for every node is implemented using a GPU kernel
- The scan operation is implemented with a CPU procedure because it's more efficient than the naive GPU counterpart
- The removal of mirrored edges is implemented using a GPU kernel
- Finding the connected components is done through a recursive GPU kernel
- Calculating the outdegrees for the supervertices is done with a GPU kernel
- The new graph allocation is done through a CPU function

In [ ]:
%%cuda_group_save --name "mstGPU.cu" --group "GPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph_d.h"
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/*****
* Device function that gets the degree of a certain node
* @param str - The structure of the graph
* @param i - The node we are interested in
*****/
__device__ node d_deg (GraphStruct *str, node i) {
    return str->cumDegs[i + 1] - str->cumDegs[i];
}

/*****
* Device function that gets the weight of a certain edge
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ int d_getWeight (GraphStruct *str, node i, uint offset) {
    return str->weights[str->cumDegs[i] + offset];
}

/*****
* Device function that gets the neighbour of a certain node
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ node d_getNeigh (GraphStruct *str, node i, uint offset) {
    return str->neighs[str->cumDegs[i] + offset];
}

__device__ uint d_getRoot (uint i, uint *d_flag, uint *d_colors) {
    return max(0, d_flag[d_colors[i]] - 1);
}

uint getRoot (uint i, uint *flag, uint *colors) {
    return max(0, flag[colors[i]] - 1);
}


/*****
* Kernel that finds the cheapest edge in the adjacency list of every node
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void findCheapest (GraphStruct *str, uint *d_candidates) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    // Initialize the minimum value
    uint minimum = UINT_MAX;
    int minimumWeight = INT_MAX;

    // Find the cheapest edge in each adjacency list
    for (uint i = 0; i < d_deg(str, idx); i++) {
        int edgeWeight = d_getWeight(str, idx, i);
        if (edgeWeight < minimumWeight) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
        else if (edgeWeight == minimumWeight &&
                 d_getNeigh(str, idx, i) < d_getNeigh(str, idx, minimum)) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
    }

    // Update the return vector
    d_candidates[idx] = minimum;
}


/*****
* Kernel that removes the mirrored edges from the graph. A mirrored edge is
* simply an edge pointing from the source to the destination and vice versa in
* an oriented graph, the removal logic is to cut the edge with the lowest source
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void mirroredEdgesRemoval (GraphStruct *str, uint *d_candidates, int *d_weight) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint destinationOffset = d_candidates[idx];
    node destination = d_getNeigh(str, idx, destinationOffset);
    if (idx < destination) {
        uint sourceOffset = d_candidates[destination];
        node destinationNeigh = d_getNeigh(str, destination, sourceOffset);

        // The vertex cannot be a candidate anymore because it would create a cycle
        if (destinationNeigh == idx) {
            d_candidates[idx] = UINT_MAX;
        }
    }

    if (d_candidates[idx] != UINT_MAX) {
        atomicAdd(d_weight, d_getWeight(str, idx, d_candidates[idx]));
    }
}


/*****
* Helper device function that recursively colors the nodes of the graph
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
* @param i - The index of the node to be colored
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__device__ uint *d_recursiveColorationHelper (GraphStruct *str, uint *d_candidates, node i, uint *d_colors) {
    uint color = UINT_MAX;
    if (d_candidates[i] == UINT_MAX) {
        color = i;
    }
    else {
        node neigh = d_getNeigh(str, i, d_candidates[i]);
        color = d_recursiveColorationHelper(str, d_candidates, neigh, d_colors)[neigh];
    }

    if (color != UINT_MAX) {
        d_colors[i] = color;
    }
    return d_colors;
}


/*****
* Kernel that recognizes the connected components in the graph and colors them
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__global__ void colorationProcess(GraphStruct *str, uint *d_candidates, uint *d_colors) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    d_recursiveColorationHelper(str, d_candidates, idx, d_colors);
}


__global__ void cumulatedDegreeUpdate(GraphStruct *str, uint *d_cumDegs, uint *d_colors, uint *d_flag) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node svSuccessor = d_getRoot(idx, d_flag, d_colors) + 1;
    uint sum = 0;

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            sum++;
        }
    }

    atomicAdd(&(d_cumDegs[svSuccessor]), sum);
}


uint *CPUScan(uint *input, uint size) {
    for (uint i = 1; i < size; i++) {
        input[i] = input[i - 1] + input[i];
    }
    return input;
}


int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine eng(FIXED_SEED);
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 1;
    Graph *graphPointer;
    Graph graph(SIZE, GPUEnabled);
    graphPointer = &graph;
  	graphPointer->randGraph(prob, true, maxWeight, eng);
    /**************************************************/


    // Checking if the random graph is connected
    if (!graphPointer->isConnected()) {
        cout << "The graph is not connected" << endl;
        return -1;
    }
    /************/


    uint iterations = 0;


    // Configuration of the GPU kernel
    uint blockDim = BLOCK_SIZE;
    uint *candidates;
    /***************/


    // Events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds;
    float spliTime = 0;
    float totalTime = 0;
    /******************/


    // Variables calculating the MST weight
    int mstWeight = 0;
    int *d_mstWeight;
    CHECK(cudaMalloc((void **)&d_mstWeight, sizeof(int)));
    CHECK(cudaMemcpy(d_mstWeight, &mstWeight, sizeof(int), cudaMemcpyHostToDevice));
    /******************************************************************************/


    // Main block of the algorithm
    while (graphPointer->getStruct()->nodeSize > 1) {
        // Initialization of the variables associated with the graph
        GraphStruct *str = graphPointer->getStruct();
        uint size = str->nodeSize;
        uint edgeSize = str->edgeSize;
        cout << "Processing a graph of size: " << size << " with " << edgeSize << " edges.\n\n";
        uint gridDim = (size + blockDim - 1) / blockDim;
        if (DEBUGGING && size < 15 && str->edgeSize < 100) {
            graphPointer->print(true);
            print_d<<<1, 1>>>(str, 1);
            CHECK(cudaDeviceSynchronize());
        }
        candidates = new uint[size];
        /******************************/

        // First setp of the algorithm
        uint *d_candidates;
        CHECK(cudaMalloc((void**)&d_candidates, (size) * sizeof(uint)));
        CHECK(cudaMemset(d_candidates, 0, (size) * sizeof(uint)));
        cout << "Launching kernel FIND CHEAPEST -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        findCheapest<<<gridDim, blockDim>>>(str, d_candidates);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Finding the cheapest edge for every vertex took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex
        if (DEBUGGING && size < 15) {
            cout << "The cheapest edge for every vertex" << endl;
            CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> " << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
            }
            cout << "\n\n\n";
        }
        /*******************/



        // Second step of the algorithm
        cout << "Launching kernel MIRRORED EDGES REMOVAL -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        mirroredEdgesRemoval<<<gridDim, blockDim>>>(str, d_candidates, d_mstWeight);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(&mstWeight, d_mstWeight, sizeof(int), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex update
        if (DEBUGGING && size < 15) {
            cout << "Update of the cheapest edge for every vertex" << endl;
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> ";
                if (candidates[i] != UINT_MAX) {
                    cout << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
                }
                else {
                    cout << "NULL" << endl;
                }
            }
            printf ("%d\n", mstWeight);
        }
        /*****************************/



        // Third step of the algorithm
        cout << "Launching kernel COLORATION PROCESS -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        // Initialize the color array
        uint *colors = new uint[size];
        uint *d_colors;
        CHECK(cudaMalloc((void**)&d_colors, size * sizeof(uint)));
        CHECK(cudaMemset(d_colors, UINT_MAX, size * sizeof(uint)));
        /**************************************************/

        cudaEventRecord(start);
        colorationProcess<<<gridDim, blockDim>>>(str, d_candidates, d_colors);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(colors, d_colors, size * sizeof(uint), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;

        // Print the coloring
        if (DEBUGGING) {
            uint *checkColoring = new uint[size];

            for (uint i = 0; i < size; i++) {
                checkColoring[i] = 0;
            }

            for (uint i = 0; i < size; i++) {
                checkColoring[colors[i]]++;
            }

            uint nonZeroColors = 0;
            for (uint i = 0; i < size; i++) {
                if (checkColoring[i] != 0) {
                    cout << "color " << i << "\t" << checkColoring[i] << endl;
                    nonZeroColors++;
                }
            }

            cout << "There is a total of " << nonZeroColors << " colors" << endl;

            cout << "\n\n\n";
        }
        /*******************/

        /**
         * If the coloring coming out of the last kernel contains only one color
         * then it means that the edge added in the last step was the one needed
         * to merge the partial trees
         **/
        uint color = colors[0];
        bool uniqueColor = true;
        for (uint i = 1; i < size; i++) {
            if (colors[i] != color) {
                uniqueColor = false;
                break;
            }
        }
        if (uniqueColor) {
            cout << "THE CALCULATION OF THE MST IS COMPLETE\n";
            cout << "THE MST WEIGHT IS: " << mstWeight << endl;
            printf("Total elapsed time: %.5f seconds\n\n", totalTime);

            // Cuda memory deallocation
            CHECK(cudaEventDestroy(start));
            CHECK(cudaEventDestroy(stop));
            CHECK(cudaFree(d_candidates));
            CHECK(cudaFree(d_colors));

            // Host memory deallocation
            delete[] candidates;
            delete[] colors;

            return 0;
        }
        /***********/



        // Fourth step of the algorithm
        cout << "Launching a round of CPU scan\n\n" << endl;
        uint *flag = new uint[size];
        for (uint i = 0; i < size; i++) {
            flag[i] = (colors[i] == i) ? 1 : 0;
        }
        cudaEventRecord(start);
        flag = CPUScan(flag, size);
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the prefix sum of the auxiliary flag array took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        if (DEBUGGING) {
            for (uint i = 0; i < size; i++) {
                if (colors[i] == i) {
                    cout << "Mapping color " << i << " to " << getRoot(i, flag, colors) << endl;
                }
            }
        }

        uint *d_flag;

        CHECK(cudaMalloc((void**)&d_flag, (size) * sizeof(uint)));
        CHECK(cudaMemcpy(d_flag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));
        /*****************************************************************************/



        // Fifth step of the algorithm

        // Allocating resources for the new cumulated degrees array
        uint newNodeSize = flag[size - 1];
        uint cumDegSize = newNodeSize + 1;
        uint *cumDegs = new uint[cumDegSize];
        uint *d_cumDegs;
        CHECK(cudaMalloc((void**)&d_cumDegs, (cumDegSize) * sizeof(uint)));
        CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));
        /***********************************/

        cout << "Launching kernel CUMULATED DEGREE UPDATE -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        cudaEventRecord(start);
        cumulatedDegreeUpdate<<<gridDim, blockDim>>>(str, d_cumDegs, d_colors, d_flag);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the computation of the cumulated degrees took: %.5f seconds\n\n", spliTime);
        CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

        // ~Debugging~ looking for errors in the cumulated degrees array
        if (DEBUGGING) {
            uint *checkDegs = new uint[cumDegSize];
            for (uint i = 0; i < cumDegSize; i++) {
                checkDegs[i] = 0;
            }
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node svSuccessor = getRoot(i, flag, colors) + 1;
                uint sum = 0;

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        sum++;
                    }
                }

                checkDegs[svSuccessor] += sum;
            }
            for (uint i = 0; i < cumDegSize; i++) {
                if (checkDegs[i] != cumDegs[i]) {
                    cout << i << " " << checkDegs[i] << " " << cumDegs[i] << endl;
                    return -1;
                }
            }
            cout << "The CPU check vector and the GPU computed one are the same" << endl;
            delete[] checkDegs;
        }
        /********************/


        // Perform another prefix sum on the cumDegrees array
        cout << "Launching a round of CPU scan\n\n" << endl;
        cudaEventRecord(start);
        cumDegs = CPUScan(cumDegs, cumDegSize);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("Doing the scan of the new cumDegs array took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;

        // ~Debugging~ print the results of the scan operation on the cum degrees array
        if (DEBUGGING) {
            uint j = 0;
            for (uint i = 0; i < cumDegSize; i++) {
                cout << cumDegs[i] << "   ";
                j++;
                if (j == 10) {
                    cout << endl;
                    j = 0;
                }
            }
        }
        /*****************/


        // Allocating space for the arrays in the newly contracted graph
        uint newEdgeSize = cumDegs[cumDegSize - 1];
        node *newNeighs = new node[newEdgeSize];
        uint *newWeights = new uint[newEdgeSize];
        /*****************/

        // Copy the contents of cumDegs into a new array
        uint *cCumDegs = new uint[cumDegSize];
        for (uint i = 0; i < cumDegSize; i++) {
            cCumDegs[i] = cumDegs[i];
        }

        cudaEventRecord(start);
        for (uint i = 0; i < size; i++) {
            uint color = colors[i];
            node superVertex = getRoot(i, flag, colors);

            for (uint j = 0; j < str->deg(i); j++) {
                node neigh = str->getNeigh(i, j);
                uint neighColor = colors[neigh];

                if (color != neighColor) {
                    int weight = str->getWeight(i, j);
                    uint position = cCumDegs[superVertex];
                    newNeighs[position] = getRoot(neigh, flag, colors);
                    newWeights[position] = weight;
                    cCumDegs[superVertex]++;
                }
            }
        }
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("The construction of the new neighbour and weight arrays took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;



        // Reconstructing the graph
        graphPointer->copyConstructor(newNodeSize, newEdgeSize, newNeighs, newWeights, cumDegs);
        printf("----------------------------------\n\n");
        /***********************************************/



        // Updating the iteration information
        totalTime += spliTime;
        iterations++;
        /*****************************************/


        // Cuda memory deallocation
        CHECK(cudaFree(d_candidates));
        CHECK(cudaFree(d_colors));
        CHECK(cudaFree(d_flag));
        CHECK(cudaFree(d_cumDegs));
        /**********************************/

        // Host memory deallocation
        delete[] candidates;
        delete[] colors;
        delete[] flag;
        delete[] cumDegs;
        delete[] newNeighs;
        delete[] newWeights;
        delete[] cCumDegs;
        /******************/
    }

    printf("Total elapsed time: %.5f seconds\n\n", totalTime);
    printf("The calculation of the MST took %d iterations\n\n", iterations);
    printf("The total weight of the tree is %d\n", mstWeight);


    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75 GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/GPU/mstGPU.cu -o mstGPU
!./mstGPU

## GPU only approach

In [ ]:
%%cuda_group_save --name "mstGPU.cu" --group "GPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph_d.h"
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/*****
* Device function that gets the degree of a certain node
* @param str - The structure of the graph
* @param i - The node we are interested in
*****/
__device__ node d_deg (GraphStruct *str, node i) {
    return str->cumDegs[i + 1] - str->cumDegs[i];
}

/*****
* Device function that gets the weight of a certain edge
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ int d_getWeight (GraphStruct *str, node i, uint offset) {
    return str->weights[str->cumDegs[i] + offset];
}

/*****
* Device function that gets the neighbour of a certain node
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ node d_getNeigh (GraphStruct *str, node i, uint offset) {
    return str->neighs[str->cumDegs[i] + offset];
}

__device__ uint d_getRoot (uint i, uint *d_flag, uint *d_colors) {
    return max(0, d_flag[d_colors[i]] - 1);
}

uint getRoot (uint i, uint *flag, uint *colors) {
    return max(0, flag[colors[i]] - 1);
}


/*****
* Kernel that finds the cheapest edge in the adjacency list of every node
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void findCheapest (GraphStruct *str, uint *d_candidates) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    // Initialize the minimum value
    uint minimum = UINT_MAX;
    int minimumWeight = INT_MAX;

    // Find the cheapest edge in each adjacency list
    for (uint i = 0; i < d_deg(str, idx); i++) {
        int edgeWeight = d_getWeight(str, idx, i);
        if (edgeWeight < minimumWeight) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
        else if (edgeWeight == minimumWeight &&
                 d_getNeigh(str, idx, i) < d_getNeigh(str, idx, minimum)) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
    }

    // Update the return vector
    d_candidates[idx] = minimum;
}


/*****
* Kernel that removes the mirrored edges from the graph. A mirrored edge is
* simply an edge pointing from the source to the destination and vice versa in
* an oriented graph, the removal logic is to cut the edge with the lowest source
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void mirroredEdgesRemoval (GraphStruct *str, uint *d_candidates, int *d_weight) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint destinationOffset = d_candidates[idx];
    node destination = d_getNeigh(str, idx, destinationOffset);
    if (idx < destination) {
        uint sourceOffset = d_candidates[destination];
        node destinationNeigh = d_getNeigh(str, destination, sourceOffset);

        // The vertex cannot be a candidate anymore because it would create a cycle
        if (destinationNeigh == idx) {
            d_candidates[idx] = UINT_MAX;
        }
    }

    if (d_candidates[idx] != UINT_MAX) {
        atomicAdd(d_weight, d_getWeight(str, idx, d_candidates[idx]));
    }
}


/*****
* Helper device function that recursively colors the nodes of the graph
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
* @param i - The index of the node to be colored
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__device__ uint *d_recursiveColorationHelper (GraphStruct *str, uint *d_candidates, node i, uint *d_colors) {
    uint color = UINT_MAX;
    if (d_candidates[i] == UINT_MAX) {
        color = i;
    }
    else {
        node neigh = d_getNeigh(str, i, d_candidates[i]);
        color = d_recursiveColorationHelper(str, d_candidates, neigh, d_colors)[neigh];
    }

    if (color != UINT_MAX) {
        d_colors[i] = color;
    }
    return d_colors;
}


/*****
* Kernel that recognizes the connected components in the graph and colors them
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__global__ void colorationProcess(GraphStruct *str, uint *d_candidates, uint *d_colors) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    d_recursiveColorationHelper(str, d_candidates, idx, d_colors);
}


/**
* Kernel that computes the prefix sun of the auxiliary flag array, code taken
* from the lectures and rearranged to follow the logic required for the
* implementation of the algorithm
* @param str - The structure of the graph
* @param d_colors - The device-level array of colors assigned to each vertex
* @param d_flag - The device-level array of flag values for the prefix sum.
*                 d_flag[i] = 1 if and only if the vertex in position i has the
*                 same color as the index => I recognize the root of the
*                 connected component
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void blockScan(uint *size, uint *input, uint *auxiliarVector) {
   __shared__ uint smem[BLOCK_SIZE];
   uint tid = threadIdx.x;
   uint idx = tid + blockIdx.x * blockDim.x;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   // Load input into shared memory.
   smem[tid] = input[idx];
   __syncthreads();

   // do recursive sums
   for (uint d = 1; d < BLOCK_SIZE; d *= 2) {
      if (tid >= d)
         smem[tid] += smem[tid - d];
      __syncthreads();
   }
   input[idx] = smem[tid];
   if (idx == ((blockIdx.x + 1) * blockDim.x - 1))
      auxiliarVector[blockIdx.x] = input[idx];
}


/**
* Auxiliary kernel that computes the sum of the partial values calculated using
* the blockScan kernel
* str - The structure of the graph
* @param d_flag - The device-level array containing intermediate sums obtained
*                 with the previous kernel
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void sumBlockScan(uint *size, uint *input, uint *auxiliarVector) {
   uint idx = threadIdx.x + blockIdx.x * blockDim.x;  // global index 0:n-1
   uint s = 0;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   if (blockIdx.x == 0) return;

   for (uint j = 0; j < blockIdx.x; j++)
      s += auxiliarVector[j];

   // add term to input
   input[idx] += s;
}


__global__ void cumulatedDegreeUpdate(GraphStruct *str, uint *d_cumDegs, uint *d_colors, uint *d_flag) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node svSuccessor = d_getRoot(idx, d_flag, d_colors) + 1;
    uint sum = 0;

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            sum++;
        }
    }

    atomicAdd(&(d_cumDegs[svSuccessor]), sum);
}


__global__ void graphContraction(GraphStruct *str, uint *d_colors, uint *d_flag,
                                 uint *d_cumDegs, node *d_newNeighs, uint *d_newWeights) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    int root = d_getRoot(idx, d_flag, d_colors);

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        if (d_colors[idx] != d_colors[neigh]) {
            int weight = d_getWeight(str, idx, i);
            uint position = atomicAdd(&(d_cumDegs[root]), 1);
            d_newNeighs[position] = d_getRoot(neigh, d_flag, d_colors);
            d_newWeights[position] = weight;
        }
    }
}


uint *CPUScan(uint *input, uint size) {
    for (uint i = 1; i < size; i++) {
        input[i] = input[i - 1] + input[i];
    }
    return input;
}


int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine eng(FIXED_SEED);
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 1;
    Graph *graphPointer;
    Graph graph(SIZE, GPUEnabled);
    graphPointer = &graph;
  	graphPointer->randGraph(prob, true, maxWeight, eng);
    /**************************************************/


    // Checking if the random graph is connected
    if (!graphPointer->isConnected()) {
        cout << "The graph is not connected" << endl;
        return -1;
    }
    /************/


    uint iterations = 0;


    // Configuration of the GPU kernel
    uint blockDim = BLOCK_SIZE;
    uint *candidates;
    /***************/


    // Events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds;
    float spliTime = 0;
    float totalTime = 0;
    /******************/


    // Variables calculating the MST weight
    int mstWeight = 0;
    int *d_mstWeight;
    CHECK(cudaMalloc((void **)&d_mstWeight, sizeof(int)));
    CHECK(cudaMemcpy(d_mstWeight, &mstWeight, sizeof(int), cudaMemcpyHostToDevice));
    /******************************************************************************/


    // Main block of the algorithm
    while (graphPointer->getStruct()->nodeSize > 1) {
        // Initialization of the variables associated with the graph
        GraphStruct *str = graphPointer->getStruct();
        uint size = str->nodeSize;
        uint edgeSize = str->edgeSize;
        cout << "Processing a graph of size: " << size << " with " << edgeSize << " edges.\n\n";
        uint gridDim = (size + blockDim - 1) / blockDim;
        if (DEBUGGING && size < 15 && str->edgeSize < 100) {
            graphPointer->print(true);
            print_d<<<1, 1>>>(str, 1);
            CHECK(cudaDeviceSynchronize());
        }
        candidates = new uint[size];
        /******************************/

        // First setp of the algorithm
        uint *d_candidates;
        CHECK(cudaMalloc((void**)&d_candidates, (size) * sizeof(uint)));
        CHECK(cudaMemset(d_candidates, 0, (size) * sizeof(uint)));
        cout << "Launching kernel FIND CHEAPEST -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        findCheapest<<<gridDim, blockDim>>>(str, d_candidates);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Finding the cheapest edge for every vertex took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex
        if (DEBUGGING && size < 15) {
            cout << "The cheapest edge for every vertex" << endl;
            CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> " << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
            }
            cout << "\n\n\n";
        }
        /*******************/



        // Second step of the algorithm
        cout << "Launching kernel MIRRORED EDGES REMOVAL -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        mirroredEdgesRemoval<<<gridDim, blockDim>>>(str, d_candidates, d_mstWeight);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(&mstWeight, d_mstWeight, sizeof(int), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex update
        if (DEBUGGING && size < 15) {
            cout << "Update of the cheapest edge for every vertex" << endl;
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> ";
                if (candidates[i] != UINT_MAX) {
                    cout << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
                }
                else {
                    cout << "NULL" << endl;
                }
            }
            printf ("%d\n", mstWeight);
        }
        /*****************************/



        // Third step of the algorithm
        cout << "Launching kernel COLORATION PROCESS -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        // Initialize the color array
        uint *colors = new uint[size];
        uint *d_colors;
        CHECK(cudaMalloc((void**)&d_colors, size * sizeof(uint)));
        CHECK(cudaMemset(d_colors, UINT_MAX, size * sizeof(uint)));
        /**************************************************/

        cudaEventRecord(start);
        colorationProcess<<<gridDim, blockDim>>>(str, d_candidates, d_colors);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(colors, d_colors, size * sizeof(uint), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;

        // Print the coloring
        if (DEBUGGING) {
            uint *checkColoring = new uint[size];

            for (uint i = 0; i < size; i++) {
                checkColoring[i] = 0;
            }

            for (uint i = 0; i < size; i++) {
                checkColoring[colors[i]]++;
            }

            uint nonZeroColors = 0;
            for (uint i = 0; i < size; i++) {
                if (checkColoring[i] != 0) {
                    cout << "color " << i << "\t" << checkColoring[i] << endl;
                    nonZeroColors++;
                }
            }

            cout << "There is a total of " << nonZeroColors << " colors" << endl;

            cout << "\n\n\n";
        }
        /*******************/

        /**
         * If the coloring coming out of the last kernel contains only one color
         * then it means that the edge added in the last step was the one needed
         * to merge the partial trees
         **/
        uint color = colors[0];
        bool uniqueColor = true;
        for (uint i = 1; i < size; i++) {
            if (colors[i] != color) {
                uniqueColor = false;
                break;
            }
        }
        if (uniqueColor) {
            cout << "THE CALCULATION OF THE MST IS COMPLETE\n";
            cout << "THE MST WEIGHT IS: " << mstWeight << endl;
            printf("Total elapsed time: %.5f seconds\n\n", totalTime);

            // Cuda memory deallocation
            CHECK(cudaEventDestroy(start));
            CHECK(cudaEventDestroy(stop));
            CHECK(cudaFree(d_candidates));
            CHECK(cudaFree(d_colors));

            // Host memory deallocation
            delete[] candidates;
            delete[] colors;

            return 0;
        }
        /***********/



        // Fourth step of the algorithm
        cout << "Launching a round of CPU scan\n\n" << endl;
        uint *flag = new uint[size];
        for (uint i = 0; i < size; i++) {
            flag[i] = (colors[i] == i) ? 1 : 0;
        }
        cudaEventRecord(start);
        flag = CPUScan(flag, size);
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the prefix sum of the auxiliary flag array took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        if (DEBUGGING) {
            for (uint i = 0; i < size; i++) {
                if (colors[i] == i) {
                    cout << "Mapping color " << i << " to " << getRoot(i, flag, colors) << endl;
                }
            }
        }

        uint *d_flag;

        CHECK(cudaMalloc((void**)&d_flag, (size) * sizeof(uint)));
        CHECK(cudaMemcpy(d_flag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));
        /*****************************************************************************/



        // Fifth step of the algorithm

        // Allocating resources for the new cumulated degrees array
        uint newNodeSize = flag[size - 1];
        uint cumDegSize = newNodeSize + 1;
        uint *cumDegs = new uint[cumDegSize];
        uint *d_cumDegs;
        CHECK(cudaMalloc((void**)&d_cumDegs, (cumDegSize) * sizeof(uint)));
        CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));
        /***********************************/

        cout << "Launching kernel CUMULATED DEGREE UPDATE -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        cudaEventRecord(start);
        cumulatedDegreeUpdate<<<gridDim, blockDim>>>(str, d_cumDegs, d_colors, d_flag);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the computation of the cumulated degrees took: %.5f seconds\n\n", spliTime);
        CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

        // ~Debugging~ looking for errors in the cumulated degrees array
        if (DEBUGGING) {
            uint *checkDegs = new uint[cumDegSize];
            for (uint i = 0; i < cumDegSize; i++) {
                checkDegs[i] = 0;
            }
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node svSuccessor = getRoot(i, flag, colors) + 1;
                uint sum = 0;

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        sum++;
                    }
                }

                checkDegs[svSuccessor] += sum;
            }
            for (uint i = 0; i < cumDegSize; i++) {
                if (checkDegs[i] != cumDegs[i]) {
                    cout << i << " " << checkDegs[i] << " " << cumDegs[i] << endl;
                    return -1;
                }
            }
            cout << "The CPU check vector and the GPU computed one are the same" << endl;
            delete[] checkDegs;
        }
        /********************/


        // Perform another prefix sum on the cumDegrees array
        cudaEventRecord(start);
        cumDegs = CPUScan(cumDegs, cumDegSize);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("Doing the scan of the new cumDegs array took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;

        // ~Debugging~ print the results of the scan operation on the cum degrees array
        if (DEBUGGING) {
            uint j = 0;
            for (uint i = 0; i < cumDegSize; i++) {
                cout << cumDegs[i] << "   ";
                j++;
                if (j == 10) {
                    cout << endl;
                    j = 0;
                }
            }
        }
        /*****************/


        // Allocating space for the arrays in the newly contracted graph
        uint newEdgeSize = cumDegs[cumDegSize - 1];
        cout << newEdgeSize << endl;
        node *newNeighs = new node[newEdgeSize];
        uint *newWeights = new uint[newEdgeSize];
        //node *d_newNeighs;
        //uint *d_newWeights;
        //CHECK(cudaMalloc((void**)&d_newNeighs, newEdgeSize * sizeof(node)));
        //CHECK(cudaMalloc((void**)&d_newWeights, newEdgeSize * sizeof(uint)));
        //CHECK(cudaMemset(d_newNeighs, 0, newEdgeSize * sizeof(node)));
        //CHECK(cudaMemset(d_newWeights, 0, newEdgeSize * sizeof(uint)));
        /*****************/

        //cout << "Launching kernel GRAPH CONTRACTION -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        //cudaEventRecord(start);
        //graphContraction<<<gridDim, blockDim>>>(str, d_colors, d_flag, d_cumDegs, d_newNeighs, d_newWeights);
        //CHECK(cudaDeviceSynchronize());
        //CHECK(cudaEventRecord(stop));
        //CHECK(cudaEventSynchronize(stop));
        //cudaEventElapsedTime(&milliseconds, start, stop);
        //printf("Generating the new vectors for the graph took: %.5f seconds\n\n", milliseconds/1000);
        //spliTime += milliseconds / 1000.0;

        //CHECK(cudaMemcpy(newNeighs, d_newNeighs, newEdgeSize * sizeof(node), cudaMemcpyDeviceToHost));
        //CHECK(cudaMemcpy(newWeights, d_newWeights, newEdgeSize * sizeof(uint), cudaMemcpyDeviceToHost));

        uint *cCumDegs = new uint[cumDegSize];

        for (uint i = 0; i < cumDegSize; i++) {
            cCumDegs[i] = cumDegs[i];
        }

        uint sum = 0;
        uint isOk= false;
        for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node superVertex = getRoot(i, flag, colors);
                if (superVertex == 0) {
                    isOk= true;
                }

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        if (isOk) {
                            sum++;
                        }
                        int weight = str->getWeight(i, j);
                        uint position = cCumDegs[superVertex];
                        newNeighs[position] = getRoot(neigh, flag, colors);
                        newWeights[position] = weight;
                        cCumDegs[superVertex]++;
                    }
                }
            }

        cout << sum << "\t" << cCumDegs[1] << endl;



        // Reconstructing the graph
        graphPointer->copyConstructor(newNodeSize, newEdgeSize, newNeighs, newWeights, cumDegs);
        //printf("Doing the complete graph contraction took: %.5f seconds\n\n", spliTime);
        //graphPointer->print(true);
        printf("----------------------------------\n\n");
        /***********************************************/



        // Updating the iteration information
        totalTime += spliTime;
        iterations++;
        /*****************************************/


        // Cuda memory deallocation
        CHECK(cudaFree(d_candidates));
        CHECK(cudaFree(d_colors));
        CHECK(cudaFree(d_flag));
        CHECK(cudaFree(d_cumDegs));
        //CHECK(cudaFree(d_newNeighs));
        //CHECK(cudaFree(d_newWeights));
        /**********************************/

        // Host memory deallocation
        delete[] candidates;
        delete[] colors;
        delete[] flag;
        delete[] cumDegs;
        delete[] newNeighs;
        delete[] newWeights;
        /******************/
    }

    printf("Total elapsed time: %.5f seconds\n\n", totalTime);
    printf("The calculation of the MST took %d iterations\n\n", iterations);
    printf("The total weight of the tree is %d\n", mstWeight);


    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}

# Kernel testing

In [ ]:
%%cuda_group_save --name "scanTesting.cu" --group "TESTING"

// Header file di C++
#include <iostream>

// Header file C
#include <time.h>
#include <cstdlib>
#include <ctime>

// Custom files
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/**
* Kernel that computes the prefix sun of the auxiliary flag array, code taken
* from the lectures and rearranged to follow the logic required for the
* implementation of the algorithm
* @param str - The structure of the graph
* @param d_colors - The device-level array of colors assigned to each vertex
* @param d_flag - The device-level array of flag values for the prefix sum.
*                 d_flag[i] = 1 if and only if the vertex in position i has the
*                 same color as the index => I recognize the root of the
*                 connected component
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void blockScan(int *size, int *input, int *auxiliarVector) {
   __shared__ uint smem[BLOCK_SIZE];
   uint tid = threadIdx.x;
   uint idx = tid + blockIdx.x * blockDim.x;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   // Load input into shared memory.
   smem[tid] = input[idx];
   __syncthreads();

   // do recursive sums
   for (uint d = 1; d < BLOCK_SIZE; d *= 2) {
      if (tid >= d) {
          __syncthreads();
          uint prev = smem[tid - d];
          smem[tid] += prev;
      }
   }
   input[idx] = smem[tid];

   if (idx == min((blockIdx.x + 1) * blockDim.x, *size) - 1) {
       auxiliarVector[blockIdx.x] = input[idx];
       //printf("%d - [%d] -- %d\n", idx, ((blockIdx.x + 1) * blockDim.x - 1), input[idx]);
       return;
   }
}


/**
* Auxiliary kernel that computes the sum of the partial values calculated using
* the blockScan kernel
* str - The structure of the graph
* @param d_flag - The device-level array containing intermediate sums obtained
*                 with the previous kernel
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void sumBlockScan(int *size, int *input, int *auxiliarVector) {
   uint idx = threadIdx.x + blockIdx.x * blockDim.x;  // global index 0:n-1
   uint s = 0;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   if (blockIdx.x == 0) return;

   for (uint j = 0; j < blockIdx.x; j++)
      s += auxiliarVector[j];

   // add term to input
   input[idx] += s;
}


int main() {
    int *input, *inputC, *inputCC;
    int size = SIZE;
    int *d_input, *d_auxiliarVector, *d_size;

    uint blockSize = BLOCK_SIZE;
    uint gridSize = (size + blockSize - 1) / blockSize;

    input = new int[size];
    inputC = new int[size];
    inputCC = new int[size];

    //int input[] = {1, 0, 1, 1, 0, 1, 0,0,0,0};
    //int inputC[] = {1, 0, 1, 1, 0, 1, 0,0,0,0};

    srand((unsigned)time(0));
    for (uint i = 0; i < size; i++) {
        input[i] = (rand()%2);
        inputC[i] = input[i];
        inputCC[i] = input[i];
    }

    if (size <= 15) {
        for (uint i = 0; i < size; i++) {
            printf("%d\n", input[i]);
        }
    }
    cout << "\n\n";

    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, 0);
    printf("Shared memory per block: %ld\n\n", deviceProp.sharedMemPerBlock);

    CHECK(cudaMalloc((void**)&d_input, size * sizeof(int)));
    CHECK(cudaMalloc((void**)&d_auxiliarVector, gridSize * sizeof(int)));
    CHECK(cudaMalloc((void**)&d_size, sizeof(int)));

    CHECK(cudaMemcpy(d_input, input, size * sizeof(int), cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_size, &size, sizeof(int), cudaMemcpyHostToDevice));
    CHECK(cudaMemset(d_auxiliarVector, 0, gridSize * sizeof(int)));

    // events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    for (uint i = 1; i < size; i++) {
        inputC[i] = inputC[i - 1] + inputC[i];
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds;
    cudaEventElapsedTime(&milliseconds, start, stop);
    float CPUtime = milliseconds / 1000.0;
    cout << "Time elapsed for CPU computation: " << CPUtime << endl;

    cout << "Block size: " << BLOCK_SIZE << endl;
    cudaEventRecord(start);
    blockScan<<<gridSize, blockSize>>>(d_size, d_input, d_auxiliarVector);
    sumBlockScan<<<gridSize, blockSize>>>(d_size, d_input, d_auxiliarVector);
    CHECK(cudaDeviceSynchronize());
    CHECK(cudaEventRecord(stop));
    CHECK(cudaEventSynchronize(stop));
    cudaEventElapsedTime(&milliseconds, start, stop);
    float GPUtime = milliseconds / 1000.0;
    cout << "Time elapsed for CPU computation: " << GPUtime << endl;

    CHECK(cudaMemcpy(input, d_input, size * sizeof(int), cudaMemcpyDeviceToHost));
    cout << "\n\n";

    //for (uint i = 0; i < size; i++) {
    //    cout << input[i] << "\t" << inputC[i] << endl;
    //}

    for (uint i = 0; i < size; i++) {
        if (input[i] != inputC[i]) {
            if (i < BLOCK_SIZE) {
                for (uint j = 0; j <= i; j++) {
                    printf("%d - GPU: %d - CPU: %d - CPUs: %d\n", j, input[j], inputC[j], inputCC[j]);
                }
                printf("THERE IS AN ERROR IN THE INSIDE CYCLE\n");
                return -1;
            }
            cout << i << "\tCPU: " << inputC[i] << ", " << " " << " GPU: " << input[i] << endl;
        }
    }

    //if (sum != input[size - 1]) {
    //    cout << "The two sums resulted in a different value, CPU: " << sum << " GPU: " << input[size - 1] << endl;
    //}
    //else {
    //    cout << "CPU: " << sum << " GPU: " << input[size - 1] << endl;
    //}
}

In [ ]:
!nvcc -arch=sm_75 src/TESTING/scanTesting.cu -o scanTesting
!./scanTesting

# Python Analysis

In [ ]:
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

resultsCPU = "/content/testing/primTest.txt"
resultsGPU = "/content/testing/gpuTest.txt"

pointerCPU = open(resultsCPU, "r")

sizes = []
times = []

bar_width = 0.35
bar_spacing = 0.5

for line in pointerCPU:
    splitLine = re.split(",", line)
    size = splitLine[0]
    time = float(splitLine[2])
    sizes.append(size)
    times.append(time)

logTimes = np.log2(times)
indices = np.arange(len(sizes)) * (1 + bar_spacing)

plt.bar(indices, logTimes, width=bar_width, edgecolor="white", linewidth=1,
        label="CPU")

pointerGPU = open(resultsGPU, "r")

sizes = []
times = []

for line in pointerGPU:
    splitLine = re.split(",", line)
    size = splitLine[0]
    time = float(splitLine[2])
    sizes.append(size)
    times.append(time)

logTimes = np.log2(times)
indices = np.arange(len(sizes)) * (1 + bar_spacing) + bar_width


plt.bar(indices, logTimes, width=bar_width, edgecolor="white", color="green",
        linewidth=1, label="GPU")
plt.xticks(indices - (bar_width / 2), sizes)

plt.legend()
plt.show()

close(pointerCPU)
close(pointerGPU)